In [1]:
# 3.3.2 テストデータを用いて評価を行う

# 既にグループ分けされたデータを使い、これまでのアプローチが正しいか評価
# 20newsgroup : 20 個のニュールグループから集められた18,828個の技術系の文書
# 各ニュースグループが各文書が属するクラスタ
# 20news-18828 をダウンロードする
# 全体の　60 % が訓練データ、40 % がテストデータとして各フォルダに分けられている

In [75]:
# ダウンロードしたデータを読み込む
import sklearn.datasets
MLCOMP_DIR = "data"
data = sklearn.datasets.load_mlcomp("20news-18828", mlcomp_root=MLCOMP_DIR)

In [76]:
# すべての文書
print(data.filenames)

[ '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/comp.graphics/1190-38614'
 '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/comp.graphics/1383-38616'
 '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/alt.atheism/487-53344'
 ...,
 '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/rec.sport.hockey/10215-54303'
 '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/sci.crypt/10799-15660'
 '/home/jovyan/work/MachineLearningSystem/Chap3/data/379/raw/comp.os.ms-windows.misc/2732-10871']


In [77]:
# 文書数
print(len(data.filenames))

18828


In [78]:
#  クラスタのラベル名
print(data.target_names)
print(len(data.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
20


In [79]:
# 訓練データを読み込む
train_data = sklearn.datasets.load_mlcomp("20news-18828", "train", mlcomp_root=MLCOMP_DIR)
print(len(train_data.filenames))

13180


In [80]:
# テストデータを読み込む
test_data = sklearn.datasets.load_mlcomp("20news-18828", "test", mlcomp_root=MLCOMP_DIR)
print(len(test_data.filenames))

5648


In [81]:
# いくつかのニュースグループに限定して取り組む
# 実験サイクル短縮

groups = [
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware',
    'comp.windows.x',
    'sci.space'
]

train_data = sklearn.datasets.load_mlcomp("20news-18828", "train", mlcomp_root=MLCOMP_DIR, categories=groups)

print("Number of training posts in tech groups:", len(train_data.filenames))

Number of training posts in tech groups: 4119


In [83]:
# 3.3.3 文書のクラスタリング

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

# 不適切な文字が含まれる可能性 => decode_error を無視するよう指定
vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5, stop_words='english', decode_error='ignore')
vectorized = vectorizer.fit_transform(train_data.data)
num_samples, num_features = vectorized.shape
print("#sample: %d, #features: %d" % (num_samples, num_features))

# 4119 の文書があり、それぞれ4751次元の特徴ベクトルを持っている

#sample: 4119, #features: 4751


In [84]:
# クラスタリング実行

# クラスタの数を 50 に設定
num_clusters = 50
from sklearn.cluster import KMeans

km = KMeans(n_clusters=num_clusters, init='random', n_init=1, verbose=1)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 6948.909
Iteration  1, inertia 3787.899
Iteration  2, inertia 3739.232
Iteration  3, inertia 3712.735
Iteration  4, inertia 3697.757
Iteration  5, inertia 3688.425
Iteration  6, inertia 3680.770
Iteration  7, inertia 3676.300
Iteration  8, inertia 3674.009
Iteration  9, inertia 3673.179
Iteration 10, inertia 3672.659
Iteration 11, inertia 3672.277
Iteration 12, inertia 3672.143
Iteration 13, inertia 3672.090
Iteration 14, inertia 3672.028
Iteration 15, inertia 3671.972
Iteration 16, inertia 3671.871
Iteration 17, inertia 3671.837
Iteration 18, inertia 3671.802
Iteration 19, inertia 3671.773
Converged at iteration 19: center shift 0.000000e+00 within tolerance 2.055311e-08


KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=50, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)

In [85]:
# それぞれの文書のラベル（どのクラスタに属すか）
print(km.labels_)

[30 24  1 ..., 36 36 11]


In [64]:
# 大きさ
print(km.labels_.shape)

(4119,)


In [86]:
# それぞれのクラスタの中心点
print(len(km.cluster_centers_))
print(km.cluster_centers_)

50
[[ 0.00617888  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.00438825 ...,  0.          0.00525286  0.        ]
 [ 0.00096087  0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.00914961  0.          0.00483236 ...,  0.          0.          0.        ]
 [ 0.          0.00165638  0.001493   ...,  0.          0.          0.        ]
 [ 0.00181061  0.00898066  0.         ...,  0.          0.          0.        ]]


In [88]:
# 3.4 最初の問題に対する答え

# 新たな文書を入力して、適切な分類ができるかどうか
new_post = 'Disk drive problems. \
Hi, I have a problem with my hard disk. \
After 1 year it is working only sporadically now. \
I tried to format it, but now it does not boot any more. \
Any ideas ? Thanks.'

# 文書をベクトル化
new_post_vec = vectorizer.transform([new_post])
new_post_label = km.predict(new_post_vec)[0]

In [89]:
# 同じクラスタに所属する文書のインデックスを取得
similar_indices = (km.labels_==new_post_label).nonzero()[0]
print(similar_indices)

[  21   97  113  184  212  267  268  272  324  333  382  413  423  539  594
  707  731  746  848  983 1045 1075 1155 1197 1217 1296 1305 1467 1527 1553
 1585 1744 1767 1794 1857 1941 2043 2087 2291 2318 2387 2457 2484 2765 2832
 2921 2966 3019 3056 3067 3077 3125 3135 3207 3316 3325 3347 3410 3480 3529
 3612 3734 3832 3873 3912 3925 3947]


In [90]:
import scipy as sp

similar = []

for i in similar_indices:
    dist = sp.linalg.norm((new_post_vec - vectorized[i]).toarray())
    similar.append((dist, train_data.data[i]))
    
similar = sorted(similar)
print(len(similar))

# 51　この文書が同じクラスタに所属していることがわかった

67


In [91]:
show_at_1 = similar[0]
show_at_2 = similar[int(len(similar)/2)]
show_at_3 = similar[-1]

print("=== #1 ===")
print(show_at_1)
print()

print("=== #2 ===")
print(show_at_2)
print()

print("=== #3 ===")
print(show_at_3)


new_post = 'Disk drive problems. \
Hi, I have a problem with my hard disk. \
After 1 year it is working only sporadically now. \
I tried to format it, but now it does not boot any more. \
Any ideas ? Thanks.'

# 似たような単語を用いた文書が得られている

=== #1 ===
(1.0203270212065101, b"From: rogntorb@idt.unit.no (Torbj|rn Rognes)\nSubject: Adding int. hard disk drive to IIcx\n\nI haven't seen much info about how to add an extra internal disk to a\nmac. We would like to try it, and I wonder if someone had some good\nadvice.\n\nWe have a Mac IIcx with the original internal Quantum 40MB hard disk,\nand an unusable floppy drive. We also have a new spare Connor 40MB\ndisk which we would like to use. The idea is to replace the broken\nfloppy drive with the new hard disk, but there seems to be some\nproblems:\n\nThe internal SCSI cable and power cable inside the cx has only\nconnectors for one single hard disk drive.\n\nIf I made a ribbon cable and a power cable with three connectors each\n(1 for motherboard, 1 for each of the 2 disks), would it work?\n\nIs the IIcx able to supply the extra power to the extra disk?\n\nWhat about terminators? I suppose that i should remove the resistor\npacks from the disk that is closest to the motherboard,

In [92]:
# 3.4.1 ノイズに対する別の見方

# 完璧なクラスタリングを期待すべきでない
# 正確に言うと、同じnewsグループの文書を同じクラスタに全て正確に所属させることを目指すべきでない

post_group = zip(train_data.data, train_data.target)

z = [(len(post[0]), post[0], train_data.target_names[post[1]]) for post in post_group]
print(sorted(z)[5:7])

# 文書を２つ取ってきた（ラベル付き）

[(90, b'Subject: E-mail of Michael Abrash?\nFrom: gmontem@eis.calstate.edu (George A. Montemayor)\n\n', 'comp.graphics'), (93, b'From: passman@world.std.com (Shirley L Passman)\nSubject: help with no docs for motherboard\n\n\n', 'comp.sys.ibm.pc.hardware')]


In [36]:
analyzer = vectorizer.build_analyzer()

# 下でprint するのは、前処理をして残った単語
# これを人間が見ても正しく分類することはできない

In [70]:
print(list(analyzer(z[5][1])))

['bharper', 'cimlinc', 'uucp', 'brett', 'harper', 'subject', 'gui', 'applic', 'framework', 'window', 'hello', 'investig', 'purchas', 'object', 'orient', 'applic', 'framework', 'come', 'look', 'good', 'zapp', 'inmark', 'zinc', 'zinc', 'softwar', 'view', 'liant', 'win', 'blais', 'consider', 'use', 'new', 'window', 'program', 'unix', 'world', 'qualiti', 'intuitiv', 'abstract', 'class', 'librari', 'provid', 'import', 'advers', 'learn', 'intern', 'window', 'program', 'new', 'program', 'methodolog', 'close', 'align', 'nativ', 'don', 'believ', 'arbitrari', 'level', 'abstract', 'just', 'sake', 'chang', 'api', 'valuabl', 'develop', '32bit', 'window', 'nt', 'memori', 'manag', 'issu', 'issu', 'particular', 'window', 'api', 'import', 'probabl', 'buy', 'class', 'librari', 'like', 'tool', 'booch', 'compon', 'ration', 'handl', 'data', 'structur', 'miscellan', 'stuff', 'alloc', 'featur', 'import', 'toolkit', 'narrow', 'zapp', 'zinc', 'toolkit', 'receiv', 'attent', 'media', 'wonder', 'hand', 'experi', 

In [38]:
print(list(analyzer(z[6][1])))

['schriejh', 'cnsvax', 'uwec', 'edu', 'subject', 'svga', 'powerbook', '160', 'ok', 'question', 'want', 'hook', 'powerbook', '160', 'svga', 'monitor', 'want', 'buy', 'powerbook', 'dos', 'companion', 'tell', 'exact', 'cabl', 'need', 'connect', 'cabl', 'purchas', 'macwharehous', 'comput', 'store', 'buy', 'cabl', 'jame', 'engin', 'run', 'hold', 'guess', 'question', 'thank', 'advanc', 'repli', 'john', 'schrieber', 'mail', 'schriejh', 'cnsvax', 'uwec', 'edu']


In [39]:
# 下２つの print は、トークン化、小文字に変換、ストップワードを除く処理をして残る単語
# また、min_df, max_df により弾かれる単語もある
# 生き残る単語はさらに少ない

print(list(set(analyzer(z[5][1])).intersection(vectorizer.get_feature_names())))

['filenam', 'esc', 'icon', 'chang', 'f1', 'brown', 'hold', 'repeat', 'sens', 'sure', 'languag', 'articl', 'return', 'box', 'easi', 'select', 'short', 'task', 'key', 'don', 'alt', 'file', 'just', 'click', 'tri', 'sub', 'mean', 'il', 'press', 'english', 'list', 'question', 'program', '1993apr22', 'use', 'group', 'learn', 'org', 'set', 'individu', 'nativ', 'assum', 'doubl', 'avoid', 'write', 'love', 'possibl', 'make', 'applic', 'like', 'system', 'road', 'answer', 'know', 'cut', 'tell', 'manag', 'tab', 'releas', 'oak', 'properti']


In [40]:
print(list(set(analyzer(z[6][1])).intersection(vectorizer.get_feature_names())))

['guess', 'cabl', 'hold', 'purchas', 'run', 'store', 'want', 'advanc', '160', 'buy', 'comput', 'engin', 'john', 'svga', 'ok', 'jame', 'repli', 'thank', 'exact', 'powerbook', 'connect', 'question', 'hook', 'monitor', 'dos', 'tell', 'mail', 'need']


In [42]:
# TF-IDF をすることで、情報を持たない単語がでてくる
# bh が最もIDF の値が高い
# 他の単語には識別性が殆ど無い

for term in ['cs', 'faq', 'thank', 'bh', 'thank']:
    print('IDF(%s)=%.2f' % (term, vectorizer._tfidf.idf_[vectorizer.vocabulary_[term]]))

IDF(cs)=3.25
IDF(faq)=4.14
IDF(thank)=2.27
IDF(bh)=6.58
IDF(thank)=2.27


In [71]:
# 3.5 パラメータの調整

# パラメータの調整をどれだけ行うべきか
# 多くの選択肢があり、自由に試すことが可能
# しかし、それが良い結果になるとは…そもそも良い結果とは何かを明確にしておく必要がある

In [ ]:
# 3.6 まとめ

# まず前処理
# 　　　　ノイズの含まれるテキストデータから、クラスタリングできる簡潔なベクトルの形へ変換
# 　　　　多くの苦労がある
# テキストの処理
#     単語の数をカウントする単純な手法が、ノイズの含まれるデータに対して有効である